In [1]:
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [2]:
from deepctr_torch.models import DeepFM

In [3]:
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names

In [4]:
data = pd.read_csv('data/criteo_sample.txt')

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']


In [5]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [6]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4 )
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                      for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [7]:
# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}

In [8]:
device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

cuda ready...


In [9]:
# 4.Define Model,train,predict and evaluate
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary',device=device)
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

In [10]:
history = model.fit(train_model_input, train[target].values,
                        batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

cpu
Train on 128 samples, validate on 32 samples, 1 steps per epoch
Epoch 1/10
0s - loss:  0.6897 - binary_crossentropy:  0.6897 - val_binary_crossentropy:  0.6806
Epoch 2/10
0s - loss:  0.6760 - binary_crossentropy:  0.6760 - val_binary_crossentropy:  0.6722
Epoch 3/10
0s - loss:  0.6623 - binary_crossentropy:  0.6623 - val_binary_crossentropy:  0.6640
Epoch 4/10
0s - loss:  0.6488 - binary_crossentropy:  0.6488 - val_binary_crossentropy:  0.6559
Epoch 5/10
0s - loss:  0.6356 - binary_crossentropy:  0.6356 - val_binary_crossentropy:  0.6482
Epoch 6/10
0s - loss:  0.6227 - binary_crossentropy:  0.6227 - val_binary_crossentropy:  0.6408
Epoch 7/10
0s - loss:  0.6099 - binary_crossentropy:  0.6099 - val_binary_crossentropy:  0.6335
Epoch 8/10
0s - loss:  0.5974 - binary_crossentropy:  0.5974 - val_binary_crossentropy:  0.6267
Epoch 9/10
0s - loss:  0.5851 - binary_crossentropy:  0.5851 - val_binary_crossentropy:  0.6201
Epoch 10/10
0s - loss:  0.5729 - binary_crossentropy:  0.5729 - val_

In [11]:
pred_ans = model.predict(test_model_input, batch_size=256)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

test LogLoss 0.6072
test AUC 0.5161
